In [1]:
#writer = pd.ExcelWriter('output.xlsx')
#occ_data.to_excel(writer,'Sheet1')
#writer.save()


import warnings
warnings.filterwarnings('ignore')

#import required Libraries
import pandas as pd
import os
import math
import numpy as np

#read in the data files
files = os.listdir("/Users/student/Desktop/Capstone/Data")

#change working directory to where the excel files are
os.chdir('C:\\Users\\student\\Desktop\\Capstone\\Data')

In [2]:
#check to make sure read in worked
for i in range(len(files)):
    print (files[i])

AsianF.xls
AsianM.xls
BlackF.xls
BlackM.xls
HawaiianF.xls
HawaiianM.xls
HispanicF.xls
HispanicM.xls
IndianF.xls
IndianM.xls
other
OtherF.xls
OtherM.xls
output.xlsx
TotalF.xls
TotalM.xls
TwoRaceF.xls
TwoRaceM.xls
WhiteF.xls
WhiteM.xls


In [3]:
#create a list of the dataset names
dataset_names=[]

for i in range(len(files)):
    dataset_names.append(files[i][:files[i].find(".")])


In [4]:
#create a dictionary with the dataset name and the actual data file
cenus_data={}

#range(len(dataset_names))
for i in range(len(dataset_names)):
    if(files[i][-3:]=="xls"):
      cenus_data[dataset_names[i]]=pd.read_excel(files[i])

    #at this point have all the occupation data sets in a dictionary with the name and then the file

In [5]:
#make sure dictionary worked
cenus_data.keys()

#pull the actual data set in the dictionary
#cenus_data[list(cenus_data.keys())[2]]

dict_keys(['AsianF', 'AsianM', 'BlackF', 'BlackM', 'HawaiianF', 'HawaiianM', 'HispanicF', 'HispanicM', 'IndianF', 'IndianM', 'OtherF', 'OtherM', 'TotalF', 'TotalM', 'TwoRaceF', 'TwoRaceM', 'WhiteF', 'WhiteM'])

In [6]:
#reassign the column names as the name of the dataset PLUS the current column name

for x in range(len(cenus_data)):
    new_col_names=[]
    temp=cenus_data[list(cenus_data.keys())[x]]

    #keep the first two columns the same name: county number and name
    #for all the other columns add the data set name infront of it
    for i in range(len(temp.columns)):
        if(i<2):
            new_col_names.append(temp.columns[i])
        else:
            new_col_names.append(list(cenus_data.keys())[x]+"_"+temp.columns[i])
    #end the loop changing column names
        
    #assign the newly created column name list to the dataset
    cenus_data[list(cenus_data.keys())[x]].columns=new_col_names

#end loop going through each dataset

In [7]:
#combine all the datasets in the cenus_data dictionary into one

occ_data=cenus_data[list(cenus_data.keys())[len(cenus_data)-1]]
    
for x in range(0,len(cenus_data)-1):
    occ_data=pd.merge(occ_data,cenus_data[list(cenus_data.keys())[x]],how="left",on="NAME")
    
#remove the duplicate county variable    
occ_data=occ_data.drop(["COUNTY_x","COUNTY_y"],axis=1)

In [8]:
#start reading in and cleaning up the income statistics dataset

In [9]:
income=pd.read_excel("/Users/student/Desktop/Capstone/Data/other/income_statistics.xls")



In [10]:
#remove first two rows
income=income.loc[2:]

#if a value in the first row is blank change it to the first value to the left
for i in range(len(income.columns)):
    if(type(income.iat[0,i])!=str and math.isnan(income.iat[0,i])):
        income.iat[0,i]=income.iat[0,i-1]
        


In [11]:
#combine the first and second row and make that the column headers
#loop through the number of columns
col_names=[]
for i in range(len(income.columns)):
    #if the second row is blank then add just the first row
    if(type(income.iat[1,i])!=str and math.isnan(income.iat[1,i])):
        col_names.append(income.iat[0,i])
    #if the second row isnot blank then combine the first and second row
    else:
        col_names.append(income.iat[0,i]+"_"+income.iat[1,i])
        
#assign the new column names
income.columns=col_names

#delete the first two rows which are now the column names
income=income.drop(income.index[:3])

In [12]:
#delete blanks, 99999 and 0 in zip code column

#delete if first column is blank
income = income[pd.notnull(income.iloc[:,0])]
#income = income[pd.notnull(income.iloc[:,1])]

#delete zip codes of 0
income=income[income.iloc[:,0] != 0]

#delete zip codes of 99999
income=income[income.iloc[:,0] != 99999]

#delete rows with a missing number of returns. This gets rid of non zip codes in first column
income = income[pd.notnull(income.iloc[:,2])]


In [13]:
#create the number of returns in each income bracket columns

one_25=income[income.iloc[:,1] == '$1 under $25,000'].iloc[:,:3]
one_25=one_25.drop("Size of adjusted gross income",axis=1)
one_25=one_25.rename(columns = {'Number of returns':'$1 under $25,000 Number of Returns'})

twentyfive_50=income[income.iloc[:,1] == '$25,000 under $50,000'].iloc[:,:3]
twentyfive_50=twentyfive_50.drop("Size of adjusted gross income",axis=1)
twentyfive_50=twentyfive_50.rename(columns = {'Number of returns':'$25,000 under $50,000 Number of Returns'})

fifty_75=income[income.iloc[:,1] == '$50,000 under $75,000'].iloc[:,:3]
fifty_75=fifty_75.drop("Size of adjusted gross income",axis=1)
fifty_75=fifty_75.rename(columns = {'Number of returns':'$50,000 under $75,000 Number of Returns'})

seventyfive_100=income[income.iloc[:,1] == '$75,000 under $100,000'].iloc[:,:3]
seventyfive_100=seventyfive_100.drop("Size of adjusted gross income",axis=1)
seventyfive_100=seventyfive_100.rename(columns = {'Number of returns':'$75,000 under $100,000 Number of Returns'})

hundo_200=income[income.iloc[:,1] == '$100,000 under $200,000'].iloc[:,:3]
hundo_200=hundo_200.drop("Size of adjusted gross income",axis=1)
hundo_200=hundo_200.rename(columns = {'Number of returns':'$100,000 under $200,000 Number of Returns'})

twohundo_up=income[income.iloc[:,1] == '$200,000 or more'].iloc[:,:3]
twohundo_up=twohundo_up.drop("Size of adjusted gross income",axis=1)
twohundo_up=twohundo_up.rename(columns = {'Number of returns':'$200,000 or more Number of Returns'})


#merge these with the income dataset to create them in columns
income=pd.merge(income,one_25,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,twentyfive_50,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,fifty_75,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,seventyfive_100,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,hundo_200,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,twohundo_up,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")



In [14]:
#delete everything besides the total row
income = income[-pd.notnull(income.iloc[:,1])]

#remove the gross income column
income=income.drop("Size of adjusted gross income", axis=1)


In [15]:
#conver all columns to ints
income=income.astype(int)

In [16]:
#match the county with the zip code

In [17]:
#load in the zip codes file
zip_codes=pd.read_excel("/Users/student/Desktop/Capstone/Data/other/zip_codes.xlsx")

In [18]:
#merge the income and zip code file together
income=pd.merge(income,zip_codes,left_on=income.iloc[:,0],right_on=zip_codes.iloc[:,0])
income=income.drop(["ZIP\ncode [1]", 'ZIP Code'],axis=1)

In [19]:
#summarize the variables by county

income=income.groupby("County",as_index=False).sum()


In [21]:
#merge the occupation and income datasets

In [22]:
#remove county from the name in occupation
for i in range(len(occ_data)):
    occ_data["NAME"][i]=occ_data["NAME"][i].replace(" County","")

In [23]:
#actually merge the occupation and income datasets

census=pd.merge(occ_data,income, left_on="NAME",right_on="County")



In [24]:
census

,NAME,WhiteM_PCT086001,WhiteM_PCT086002,WhiteM_PCT086003,WhiteM_PCT086004,WhiteM_PCT086005,WhiteM_PCT086006,WhiteM_PCT086007,WhiteM_PCT086008,WhiteM_PCT086009,...,Tax due at time of filing [11]_Number of returns,Tax due at time of filing [11]_Amount,Overpayments refunded [12]_Number of returns,Overpayments refunded [12]_Amount,"$1 under $25,000 Number of Returns","$25,000 under $50,000 Number of Returns","$50,000 under $75,000 Number of Returns","$75,000 under $100,000 Number of Returns","$100,000 under $200,000 Number of Returns","$200,000 or more Number of Returns"
0,Alamance,50409,26484,7433,3813,2981,650,203,169,522,...,13290,48075,58020,150599,28880,19790,10120,6220,7190,1500
1,Alexander,16976,9243,1558,907,730,132,40,22,70,...,2330,7870,11610,29925,5810,4060,2190,1220,1000,170
2,Alleghany,4883,2698,619,391,367,50,4,4,41,...,880,2938,3570,8976,2130,1250,590,370,340,30
3,Anson,5754,3139,574,335,298,39,8,14,39,...,1350,4623,7640,21507,4500,2700,990,550,480,50
4,Ashe,11089,6085,1213,686,579,91,2,7,62,...,2250,7456,8160,19440,4800,2800,1480,880,850,130
5,Avery,6979,3709,811,496,414,47,13,12,27,...,1570,4851,5220,11864,3150,1900,970,510,500,120
6,Beaufort,14300,7860,1953,948,766,177,50,11,92,...,4170,15632,15750,42501,9010,5100,2610,1720,2080,380
7,Bertie,3391,1788,518,335,304,53,6,3,19,...,1220,3710,6180,17610,3680,2250,910,400,420,30
8,Bladen,8352,4550,1159,645,474,84,0,12,50,...,1800,6079,8980,25248,4950,3210,1330,760,750,100
9,Brunswick,27006,14971,3031,1692,1414,274,113,32,189,...,13910,58804,40890,108478,20350,12870,7970,6050,8660,2170
